In [1]:
import openpathsampling as paths
import numpy as np
import openpathsampling.engines.openmm as peng

Load first frame from test pdb.

In [2]:
template = peng.snapshot_from_pdb('../data/Alanine_solvated.pdb')

Create a simple CV without an underlying function.

In [3]:
cv0 = paths.CollectiveVariable('func0')

Create a complicated function.

In [4]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [5]:
cv1 = paths.CV_Function('func1', dist, center=center, np=np)
cv2 = paths.CV_Function('func2', dist, center=center, np=np, cv_wrap_numpy_array=True)
cv3 = paths.CV_Function('func3', dist, center=center, np=np, cv_wrap_numpy_array=True, cv_time_reversible=True)

Create storage to test save and load.

In [6]:
storage = paths.Storage('can_be_deleted.nc', mode='w', template=template)

Save CV

In [7]:
#! lazy
print storage.save([cv0, cv1, cv2, cv3])

[(store.cvs[CollectiveVariable], 6, 0), (store.cvs[CollectiveVariable], 6, 1), (store.cvs[CollectiveVariable], 6, 2), (store.cvs[CollectiveVariable], 6, 3)]


Set the CV value for the storage.template

In [8]:
cv0[template] = 10.0

and create a storage for it

In [9]:
storage.cvs.create_cache(cv0)

And we should have a float store

In [10]:
cv0_fw = storage.cvs.cache_var(cv0)
cv0_bw = storage.cvs.cache_bw(cv0)

In [11]:
print cv0_fw
print cv0_bw

<type 'netCDF4._netCDF4.Variable'>
float32 snapshots_cv_0_values_fw(snapshots)
    var_type: float
    maskable: True
unlimited dimensions: snapshots
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used

<type 'netCDF4._netCDF4.Variable'>
float32 snapshots_cv_0_values_bw(snapshots)
    var_type: float
    maskable: True
unlimited dimensions: snapshots
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used



In [12]:
assert(cv0_fw.var_type == 'float')
assert(cv0_bw.var_type == 'float')

Check, if the value had been stored correctly.

In [13]:
print cv0_fw[:]
print cv0_bw[:]

[None]
[None]


Sync, and then it should work.

In [14]:
cv0.sync()

In [15]:
assert(cv0_fw[:] == [10.0])
assert(cv0_bw[:] == [None])

In [16]:
print cv0._cache_dict.cache._cache.values()

[10.0]


Test function for reversed template.

In [17]:
dd = dist(template.reversed, center, np)
print dd

-1.08320000023


Create another CV. This time using the `from_template` function.

In [18]:
print cv0([template, template])
print cv1([template, template])

[10.0, 10.0]
[-1.0832000002264977, -1.0832000002264977]


In [19]:
print type(cv0([template, template]))
print type(cv0([template, template])[0])
print type(cv1([template, template]))
print type(cv1([template, template])[0])

<type 'list'>
<type 'float'>
<type 'list'>
<type 'numpy.float64'>


In [20]:
#! skip
print storage.cvs.variables['json'][:]

[ u'{"_cls": "CollectiveVariable", "_dict": {"cv_store_cache": false, "cv_time_reversible": false, "name": "func0"}}'
 u'{"_cls": "CV_Function", "_dict": {"name": "func1", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTQtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB", "_module_vars": [], "_global_vars": []}, "cv_store_cache": true, "cv_time_reversible": false, "cv_wrap_numpy_array": false, "cv_requires_lists": false, "cv_scalarize_numpy_singletons": false, "kwargs": {"np": {"_import": "numpy"}, "center": 1}}}'
 u'{"_cls": "CV_Function", "_dict": {"name": "func2", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucH

In [21]:
cv0j = storage.cvs.vars['json'][0]
cv1j = storage.cvs.vars['json'][1]
cv2j = storage.cvs.vars['json'][2]
cv3j = storage.cvs.vars['json'][3]

In [22]:
res = cv0j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv1j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv2j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv3j([template, template, template])
assert(res[0] == res[1] == res[2])

In [23]:
for store in [
    'snapshots_cv_0_values_fw', 
    'snapshots_cv_0_values_bw', 
    'snapshots_cv_1_values_fw', 
    'snapshots_cv_1_values_bw', 
    'snapshots_cv_2_values_fw', 
    'snapshots_cv_2_values_bw', 
    'snapshots_cv_3_values_fw'
]:
    assert(store in storage.variables)
    
assert('snapshots_cv_3_values_bw' not in storage.variables)

In [24]:
print storage.cvs.cache_var(0)[:]
print storage.cvs.cache_var(1)[:]
print storage.cvs.cache_var(2)[:]
print storage.cvs.cache_var(3)[:]

[10.0]
[None]
[None]
[None]


Sync again.

In [25]:
cv0.sync()
cv1.sync()
cv2.sync()
cv3.sync()

Note that the third CV, which had `time_reversible=True` will automatically compute the values for the reversed snapshot, too. Hence it has no additional dictionary for the reversed values. After syncing we see two equal values.

In [26]:
print storage.cvs.cache_var(0)[:]
print storage.cvs.cache_bw(0)[:]
print storage.cvs.cache_var(1)[:]
print storage.cvs.cache_bw(1)[:]
print storage.cvs.cache_var(2)[:]
print storage.cvs.cache_bw(2)[:]
print storage.cvs.cache_var(3)[:]

[10.0]
[None]
[-1.0832000002264977]
[None]
[-1.0832000002264977]
[None]
[-1.0832000002264977]


Since our function returns a numpy.float64 scalar we get these back (hopefully)

In [27]:
print type(storage.vars['snapshots_cv_0_values_fw'][0])
print type(storage.vars['snapshots_cv_1_values_fw'][0])
print type(storage.vars['snapshots_cv_2_values_fw'][0])
print type(storage.vars['snapshots_cv_3_values_fw'][0])

<type 'float'>
<type 'numpy.float64'>
<type 'numpy.float64'>
<type 'numpy.float64'>


In [28]:
print cv0._store_dict.cache
print cv1._store_dict.cache
print cv2._store_dict.cache
print cv3._store_dict.cache

LRUCache(1/0 of 1000000/0)
LRUCache(1/0 of 1000000/0)
LRUCache(1/0 of 1000000/0)
LRUCache(2/0 of 1000000/0)
